# Tweepy

In [10]:
import tweepy
consumer_key = ''
consumer_secret = ''
access_key= ''
access_secret = ''

start_date = '2020-03-20'
end_date = '2020-03-20'

#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

a = tweepy.Cursor(api.search, q='PSBB',
              count=1, include_rts=False, since=start_date, tweet_mode="extended").pages(1)
for page in a:
    for status in page:
        public_tweets = status
        
import json 

status = public_tweets

#convert to string
json_str = json.dumps(status._json)

#deserialise string into python object
parsed = json.loads(json_str)

b = json.dumps(parsed, indent=4, sort_keys=True)

# Slang

In [53]:
import ast
file_2  = open("cleaning_source/combined_slang_words.txt", "r")
content2 = file_2.read()
slang_words = ast.literal_eval(content2)
file_2.close()

In [54]:
len(slang_words)

1018

In [55]:
slang_words['gak']  = 'tidak'
slang_words['trus'] = 'lalu'
slang_words['dlm'] = 'dalam'
slang_words['tetep'] = 'tetap'
slang_words['skrg'] = 'sekarang'
slang_words['sm'] = 'dengan'
slang_words['udh'] = 'sudah'
slang_words['cm'] = 'cuma'
slang_words['org'] = 'orang'
slang_words['bangor'] = 'nakal'
slang_words['ngamuk'] = 'mengamuk'
slang_words['iso'] = 'bisa'
slang_words['mbuh'] = 'tidak tahu'
slang_words['kzl'] = 'kesal'
slang_words['name'] = 'nama'


In [56]:
f = open("cleaning_source/update_combined_slang_words.txt","w")
f.write( str(slang_words) )
f.close()

In [57]:
import ast
file_2  = open("cleaning_source/update_combined_slang_words.txt", "r")
content2 = file_2.read()
update_slang_words = ast.literal_eval(content2)
file_2.close()

In [58]:
len(update_slang_words)

1033

# Lexicon

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import pytz 
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
pos_lexicon = pd.read_csv('lexicon/InSet-master/positive.tsv',sep='\t')
neg_lexicon = pd.read_csv('lexicon/InSet-master/negative.tsv',sep='\t')

In [3]:
pos_lexicon

,word,weight
0,hai,3
1,merekam,2
2,ekstensif,3
3,paripurna,1
4,detail,2
...,...,...
3604,melarikan,3
3605,vakansi,3
3606,spesial,4
3607,asrama,3


In [4]:
neg_lexicon

,word,weight
0,putus tali gantung,-2
1,gelebah,-2
2,gobar hati,-2
3,tersentuh (perasaan),-1
4,isak,-5
...,...,...
6604,kantong kering,-4
6605,penggaruk,-3
6606,senewen,-4
6607,menetapkan,-5


In [5]:
lexicon = pos_lexicon.append(neg_lexicon,ignore_index=True)

In [6]:
len(lexicon)

10218

In [7]:
addition = pd.read_csv('lexicon/agusmakmun/sentimentword.csv')

In [8]:
addition.isnull().sum()

word      0
weight    0
dtype: int64

In [9]:
lexicon.isnull().sum()

word      0
weight    0
dtype: int64

In [10]:
lexicon_word = lexicon['word'].to_list()

In [11]:
lexicon['word'][0] in lexicon_word

True

In [12]:
add_word = []
add_weight = []
for i in range(0,len(addition)):
    if (addition['word'][i] not in lexicon_word):
        add_word.append(addition['word'][i])
        add_weight.append(addition['weight'][i])

addition_lexicon = pd.DataFrame(list(zip(add_word,add_weight)),columns =['word','weight'])

In [13]:
addition_lexicon

,word,weight
0,acuh,-1
1,agresi,-3
2,ajaib,3
3,akal,2
4,alarm,-3
...,...,...
821,waktunya,-2
822,waspadalah,-2
823,wow,3
824,wtf,-3


In [14]:
full_lexicon = lexicon.append(addition_lexicon,ignore_index = True)

In [15]:
full_lexicon.sample(5)

,word,weight
5452,bedah,-3
10857,pendiri,-2
6468,kepalang,-3
6104,alami,-3
9110,ringkik,-3


In [16]:
full_lexicon[full_lexicon['weight']<0].min()

word      (barang) bekas
weight                -5
dtype: object

In [17]:
my_file = open("lexicon/swearwords/swear-words.txt", "r")
content = my_file.read()
swear_words = content.split("\n")

In [18]:
weight_swear = [-5 for i in range(len(swear_words))]

In [19]:
swear_lexicon = pd.DataFrame(list(zip(swear_words,weight_swear)),columns =['word','weight'])

In [20]:
swear_lexicon

,word,weight
0,anjing,-5
1,anjiang,-5
2,anjir,-5
3,anjay,-5
4,asu,-5
...,...,...
64,sontoloyo,-5
65,tai,-5
66,telek,-5
67,tolol,-5


In [21]:
full_lexicon = lexicon.append(swear_lexicon,ignore_index = True)

In [22]:
len(full_lexicon)

10287

In [26]:
full_lexicon

,word,weight
0,hai,3
1,merekam,2
2,ekstensif,3
3,paripurna,1
4,detail,2
...,...,...
10282,sontoloyo,-5
10283,tai,-5
10284,telek,-5
10285,tolol,-5


In [28]:
def number_of_words(x):
    words = word_tokenize(x['word'])
    number = len(words)
    return number 

In [30]:
full_lexicon['number_of_words'] = full_lexicon.apply(lambda x: number_of_words(x),axis=1)

In [42]:
full_lexicon = full_lexicon.drop(full_lexicon[full_lexicon['number_of_words'] == 0].index[0],axis=0)

In [43]:
full_lexicon  = full_lexicon.reset_index(drop=True)

In [48]:
full_lexicon[full_lexicon['number_of_words'] == 3]

,word,weight,number_of_words
117,jalan bebas hambatan,2,3
364,asah asih asuh,2,3
574,lunas (kapal,2,3
1106,mencium telapak kaki,1,3
1501,naik ke pelaminan,3,3
1853,naik ke pelamin,3,3
1871,cium telapak kaki,1,3
1904,minta kpd tuhan,3,3
1927,jalan bebas hambat,2,3
2073,acuh tak acuh,4,3


In [46]:
full_lexicon.to_csv(r'lexicon\full_lexicon.csv',index=False)

In [41]:
full_lexicon

10286

In [77]:
modified_lexicon = pd.read_csv('lexicon/modified_full_lexicon.csv')

In [50]:
modified_lexicon['number_of_words'] = modified_lexicon.apply(lambda x: number_of_words(x),axis=1)

In [52]:
modified_lexicon.to_csv(r'lexicon\modified_full_lexicon.csv',index=False)

# Stopwords

In [91]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [92]:
my_file = open("cleaning_source/combined_stop_words.txt", "r")
content = my_file.read()
stop_words = content.split("\n")

In [93]:
lexicon_word = modified_lexicon['word'].to_list()

In [94]:
overlap=[]
for word in stop_words:
    if word in lexicon_word:
        overlap.append(word)
    else:
        kata_dasar = stemmer.stem(word)
        if kata_dasar in lexicon_word:
            overlap.append(word)
    

In [95]:
overlap

['akulah',
 'amatlah',
 'andalah',
 'apa',
 'artinya',
 'awalnya',
 'bagi',
 'balik',
 'begini',
 'beginikah',
 'bekerja',
 'benarkah',
 'berada',
 'berarti',
 'berbagai',
 'beri',
 'berikut',
 'berkata',
 'berkeinginan',
 'berlainan',
 'berlangsung',
 'bermaksud',
 'bersama',
 'bersiap',
 'bertanya',
 'berturut',
 'bertutur',
 'berupa',
 'betul',
 'biasa',
 'boleh',
 'bolehlah',
 'bulan',
 'cukup',
 'cukuplah',
 'dahulu',
 'dari',
 'datang',
 'dia',
 'diberi',
 'diberikannya',
 'dibuatnya',
 'didatangkan',
 'diingat',
 'dijelaskan',
 'dikarenakan',
 'dikatakannya',
 'diketahui',
 'dikira',
 'dilalui',
 'dimaksud',
 'dimaksudkannya',
 'diminta',
 'dimulailah',
 'dimungkinkan',
 'dipastikan',
 'diperbuatnya',
 'diperkirakan',
 'diperlukan',
 'dipersoalkan',
 'dipunyai',
 'dirinya',
 'disinilah',
 'ditanyai',
 'dituturkannya',
 'diucapkannya',
 'entah',
 'guna',
 'hanya',
 'haruslah',
 'hendak',
 'hendaknya',
 'ikut',
 'ingat-ingat',
 'itukah',
 'jadi',
 'jadinya',
 'jauh',
 'jawaban',
 